In [166]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('ggplot')

import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import  accuracy_score

In [167]:
#  ファイルの読み込み]
DATA_HOME_DIR = "./raw/"
train = pd.read_csv(DATA_HOME_DIR + 'train.csv')
test = pd.read_csv(DATA_HOME_DIR + 'test.csv')
gender_submission = pd.read_csv(DATA_HOME_DIR + 'gender_submission.csv')

前処理

In [168]:
# 前処理のためにtrainとtestを連結
train['is_test'] = 0
train['is_train'] = 1
test['is_test'] = 1 
test['is_train'] = 0

# 目的変数
Y = train['Survived']
train_X = train.drop(['Survived'], axis = 1)

# テストID
test_id = test['PassengerId']
test_X = test

# 連結
all_df = pd.concat([train_X, test_X], axis=0)

In [169]:
# 欠損値}
all_df.isnull().sum()

PassengerId       0
Pclass            0
Name              0
Sex               0
Age             263
SibSp             0
Parch             0
Ticket            0
Fare              1
Cabin          1014
Embarked          2
is_test           0
is_train          0
dtype: int64

In [170]:
# Age欠損値処理
# 男女別でPclass毎の中央値で埋め
# all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mean())   # 平均値
# all_df['Age'] = all_df['Age'].fillna(all_df['Age'].median()) # 中央値
# all_df['Age'] = all_df['Age'].fillna(all_df['Age'].mode())   # 最頻値

m1 = all_df[(all_df['Sex'] == 'male') & (all_df['Pclass'] == 1)]['Age'].median()
m2 = all_df[(all_df['Sex'] == 'male') & (all_df['Pclass'] == 2)]['Age'].median()
m3 = all_df[(all_df['Sex'] == 'male') & (all_df['Pclass'] == 3)]['Age'].median()
f1 = all_df[(all_df['Sex'] == 'female') & (all_df['Pclass'] == 1)]['Age'].median()
f2 = all_df[(all_df['Sex'] == 'female') & (all_df['Pclass'] == 2)]['Age'].median()
f3 = all_df[(all_df['Sex'] == 'female') & (all_df['Pclass'] == 3)]['Age'].median()

all_df.loc[(all_df['Age'].isnull()) & (all_df['Sex'] == 'male') & (all_df['Pclass'] == 1),  'Age'] = m1
all_df.loc[(all_df['Age'].isnull()) & (all_df['Sex'] == 'male') & (all_df['Pclass'] == 2),  'Age'] = m2
all_df.loc[(all_df['Age'].isnull()) & (all_df['Sex'] == 'male') & (all_df['Pclass'] == 3),  'Age'] = m3
all_df.loc[(all_df['Age'].isnull()) & (all_df['Sex'] == 'female') & (all_df['Pclass'] == 1),  'Age'] = f1
all_df.loc[(all_df['Age'].isnull()) & (all_df['Sex'] == 'female') & (all_df['Pclass'] == 2),  'Age'] = f2
all_df.loc[(all_df['Age'].isnull()) & (all_df['Sex'] == 'female') & (all_df['Pclass'] == 3),  'Age'] = f3

In [171]:
# Fare欠損値確認
all_df[all_df['Fare'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,is_train
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S,1,0


In [172]:
# Fare欠損値処理
# Pcalass=3の平均値で埋め
Pclass3mean = all_df[all_df['Pclass'] == 3]['Fare'].mean()
all_df['Fare'].fillna(Pclass3mean, inplace=True) 

In [173]:
all_df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,is_train
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1
5,6,3,"Moran, Mr. James",male,25.0,0,0,330877,8.4583,NaN,Q,0,1
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0,1
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,0,1
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,0,1
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,0,1


In [174]:
# Embarked欠損値確認
all_df[all_df['Embarked'].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,is_train
61,62,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,0,1
829,830,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,0,1


In [175]:
# pclass=1の人が割合が一番多いCで埋める
all_df['Embarked'].fillna('C', inplace=True) 

In [176]:
all_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,is_train
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1


In [177]:
#  Ageのカテゴリ変数化
# all_df.loc[ all_df['Age']  <=  10,  'Age_c']  =  0
# all_df.loc[(all_df['Age'] >  10)  &  (all_df['Age']  <=  20),  'Age_c']  =  1
# all_df.loc[(all_df['Age'] >  40)  &  (all_df['Age']  <=  30),  'Age_c']  =  2
# all_df.loc[(all_df['Age'] >  30)  &  (all_df['Age']  <=  40),  'Age_c']  =  3
# all_df.loc[(all_df['Age'] >  40)  &  (all_df['Age']  <=  50),  'Age_c']  =  4
# all_df.loc[(all_df['Age'] >  50)  &  (all_df['Age']  <=  60),  'Age_c']  =  5
# all_df.loc[(all_df['Age'] >  60)  &  (all_df['Age']  <=  70),  'Age_c']  =  6
# all_df.loc[(all_df['Age'] >  70)  &  (all_df['Age']  <=  80),  'Age_c']  =  7
# all_df.loc[ all_df['Age']  >  80,  'Age_c']    =  8

変数追加

In [178]:
all_df['AgeBin'] = pd.cut(all_df['Age'].astype(int), 5)
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
all_df['AgeBin'] = label.fit_transform(all_df['AgeBin'])

In [179]:
all_df['FareBin'] = pd.cut(all_df['Fare'].astype(int), 5)
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
all_df['FareBin'] = label.fit_transform(all_df['FareBin'])

In [180]:
# 家族数 (自分を含める)
all_df['CountFamily'] = all_df['SibSp'] + all_df['Parch'] + 1

In [181]:
# 一人フラグ
all_df['IsHitori'] = 0
all_df.loc[all_df['CountFamily'] == 1,  'IsHitori'] = 1

In [182]:
# Cabin の先頭一文字抽出
all_df['Cabin_initial'] = all_df['Cabin'].str[:1]
all_df['Cabin_initial'].value_counts()

C    94
B    65
D    46
E    41
A    22
F    21
G     5
T     1
Name: Cabin_initial, dtype: int64

In [183]:
all_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_test,is_train,AgeBin,FareBin,CountFamily,IsHitori,Cabin_initial
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,1,1,0,2,0,NaN
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,1,2,0,2,0,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,1,1,0,1,1,NaN
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,1,2,0,2,0,C
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,1,2,0,1,1,NaN


In [184]:
all_df.loc[all_df['Sex'] == 'male', 'Sex'] = 0
all_df.loc[all_df['Sex'] == 'female', 'Sex'] = 1
all_df['Sex'].astype(int)

0      0
1      1
2      1
3      1
4      0
5      0
6      0
7      0
8      1
9      1
10     1
11     1
12     0
13     0
14     1
15     1
16     0
17     0
18     1
19     1
20     0
21     0
22     1
23     0
24     1
25     1
26     0
27     0
28     1
29     0
      ..
388    0
389    0
390    0
391    1
392    0
393    0
394    0
395    1
396    0
397    1
398    0
399    0
400    1
401    0
402    1
403    0
404    0
405    0
406    0
407    0
408    1
409    1
410    1
411    1
412    1
413    0
414    1
415    0
416    0
417    0
Name: Sex, Length: 1309, dtype: int32

In [185]:
# nameの敬称抽出
import collections 
collections.Counter(all_df['Name'].str.extract('([A-Za-z]+)\.',  expand = False))

Counter({'Capt': 1,
         'Col': 4,
         'Countess': 1,
         'Don': 1,
         'Dona': 1,
         'Dr': 8,
         'Jonkheer': 1,
         'Lady': 1,
         'Major': 2,
         'Master': 61,
         'Miss': 260,
         'Mlle': 2,
         'Mme': 1,
         'Mr': 757,
         'Mrs': 197,
         'Ms': 2,
         'Rev': 8,
         'Sir': 1})

In [186]:
# df_trainのTitle列の値ごとに平均値を算出
all_df['Title']=  all_df['Name'].str.extract('([A-Za-z]+)\.', expand=False)
all_df.groupby('Title').mean()['Age']

Title
Capt        70.000000
Col         54.000000
Countess    33.000000
Don         40.000000
Dona        39.000000
Dr          43.375000
Jonkheer    38.000000
Lady        48.000000
Major       48.500000
Master       8.042295
Miss        21.917654
Mlle        24.000000
Mme         24.000000
Mr          31.249670
Mrs         35.680203
Ms          25.000000
Rev         41.250000
Sir         49.000000
Name: Age, dtype: float64

In [187]:
#  変換関数
def title_to_c(title):
    if title == 'Master':
        return  'Master'
    elif title == 'Miss':
        return  'Miss'
    elif title == 'Mr':
        return  'Mr'
    elif title == 'Mrs':
        return  'Mrs'
    else:
        return  'other'
#  リスト内包表記を用いて変換
all_df['Title'] = [title_to_c(i) for i in all_df['Title']]

In [188]:
# 苗字の取り出し
all_df['FamilyName'] = all_df['Name'].str.extract('([A-Za-z]+)\,', expand=False)

In [189]:
# Ticket の先頭一文字抽出
all_df['Ticket_initial'] = all_df['Ticket'].str[:1]
all_df['Ticket_initial'].value_counts()

3    429
2    278
1    210
P     98
S     98
C     77
A     42
W     19
F     13
7     13
4     11
6      9
L      5
5      3
8      2
9      2
Name: Ticket_initial, dtype: int64

In [190]:
all_df.loc[all_df['Ticket_initial'].isin(['1', '2', '3' ,'4' ,'5' ,'6' ,'7' ,'8' ,'9' ,'0']), 'Ticket_initial'] = 0
all_df.loc[all_df['Ticket_initial'] != 0, 'Ticket_initial'] = 1
all_df['Ticket_initial'].value_counts()

0    957
1    352
Name: Ticket_initial, dtype: int64

In [191]:
all_df.drop(['Name', 'Cabin', 'Ticket', 'SibSp', 'Parch', 'Age', 'Fare'],  axis=1,  inplace=True)

In [192]:
# カテゴリ変数を取得する関数
def _get_categorical_features(df):
    feats = [col for col in list(df.columns) if df[col].dtype == 'object']
    return feats

# カテゴリ変数をファクトライズ (整数に置換)する関数
def _factorize_categoricals(df, cats):
    for col in cats:
        df[col], _ = pd.factorize(df[col])
    return df 

# カテゴリ変数のダミー変数 (二値変数化)を作成する関数
def _get_dummies(df, cats):
    for col in cats:
        df = pd.concat([df, pd.get_dummies(df[col], prefix=col)], axis=1)
    return df 

In [193]:
# カテゴリ変数を取得
all_df_cats = _get_categorical_features(all_df)
all_df_cats

['Embarked', 'Cabin_initial', 'Title', 'FamilyName']

In [194]:
# ダミー変数を取得
all_df = _get_dummies(all_df, all_df_cats)
all_df.drop(all_df_cats,  axis=1,  inplace=True)
all_df.head()

,PassengerId,Pclass,Sex,is_test,is_train,AgeBin,FareBin,CountFamily,IsHitori,Ticket_initial,...,FamilyName_Yasbeck,FamilyName_Young,FamilyName_Youseff,FamilyName_Yousif,FamilyName_Yousseff,FamilyName_Yrois,FamilyName_Zabour,FamilyName_Zakarian,FamilyName_Zimmerman,FamilyName_hoef
0,1,3,0,0,1,1,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,1,2,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0
2,3,3,1,0,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,4,1,1,0,1,2,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,3,0,0,1,2,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [195]:
# 最終的なtrainとtestデータを用意
ignore_features = ['PassengerId', 'is_train', 'is_test']
relevant_features = [col for col in all_df.columns if col not in ignore_features]
trainX = all_df[all_df['is_train'] == 1][relevant_features]
testX = all_df[all_df['is_test'] == 1][relevant_features]

In [196]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
# LogisticRegressionでグリッドサーチ
parameters = {
        'C'      : [ 50.0, 60.0, 70.0, 80.0, 100.0, 150.0],
        'random_state'      : [17]
}
lr = GridSearchCV(LogisticRegression(), parameters, cv=3)
lr.fit(trainX, Y)

print('LogisticRegression Best parameters: {}'.format(lr.best_params_))
print('LogisticRegression Best score: {:.3f}'.format(lr.best_score_))

predictions = lr.predict(testX)

C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

LogisticRegression Best parameters: {'C': 50.0, 'random_state': 17}
LogisticRegression Best score: 0.817


In [208]:
# ベイズ最適化でランダムフォレストのパラメータ
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
def randomforest_cv(n_estimators, min_samples_split, max_features):
    val = cross_val_score(
        RandomForestClassifier(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max_features,
            random_state=0
        ),
        trainX, Y,
        scoring = 'accuracy',
        cv = 3, # 3-fold
        n_jobs = -1 # use all CPUs
    ).mean()
    return val

randomforest_cv_bo = BayesianOptimization(
    randomforest_cv,
    {'n_estimators': (10, 250),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999)}
)

gp_params = {"alpha": 1e-5}
randomforest_cv_bo.maximize(n_iter=50, **gp_params)
print(randomforest_cv_bo.res['max']['max_val'])
print(randomforest_cv_bo.res['max']['max_params'])

Initialization
-------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_features |   min_samples_split |   n_estimators | 
    1 | 00m00s |    0.82828 |         0.7640 |             19.6340 |       146.3497 | 
    2 | 00m00s |    0.83614 |         0.9575 |              5.8005 |        40.2249 | 
    3 | 00m00s |    0.82941 |         0.6098 |              3.1427 |       118.6193 | 
    4 | 00m00s |    0.82941 |         0.2643 |              5.1949 |        95.4375 | 
    5 | 00m00s |    0.82267 |         0.9662 |             22.8243 |        86.9264 | 
Bayesian Optimization
-------------------------------------------------------------------------------------
 Step |   Time |      Value |   max_features |   min_samples_split |   n_estimators | 
    6 | 00m03s |    0.81594 |         0.1588 |              2.1000 |        10.0312 | 
    7 | 00m04s |    0.82604 |         0.4677 |              2.2445 |       249.9487 | 
    8 | 

C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([2.53749373e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 5, 'warnflag': 2}
  ConvergenceWarning)


   33 | 00m05s |    0.83502 |         0.3818 |              2.4399 |        10.0016 | 
   34 | 00m05s |    0.83502 |         0.8499 |              2.0755 |        10.0500 | 
   35 | 00m05s |    0.81818 |         0.6463 |              2.0771 |        10.0924 | 
   36 | 00m05s |    0.82828 |         0.4789 |              2.4874 |        10.0509 | 
   37 | 00m05s |    0.82604 |         0.4119 |              2.5570 |        10.0348 | 
   38 | 00m05s |    0.82379 |         0.6686 |              2.1014 |        10.0387 | 
   39 | 00m05s |    0.82604 |         0.6432 |             24.9693 |        10.2116 | 
   40 | 00m06s |    0.83389 |         0.7319 |              2.1303 |        10.0105 | 
   41 | 00m06s |    0.83389 |         0.6913 |              2.0073 |        10.2950 | 
   42 | 00m06s |    0.82828 |         0.4375 |              2.1366 |        10.1974 | 
   43 | 00m06s |    0.81818 |         0.3673 |              2.4525 |        10.0904 | 
   44 | 00m06s |    0.81818 |         0.236

C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-7.04062429e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 46, 'nit': 4, 'warnflag': 2}
  ConvergenceWarning)


   53 | 00m06s |    0.83277 |         0.7191 |              2.0561 |        10.0284 | 


C:\Users\ykcg2\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.0001633]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 46, 'nit': 4, 'warnflag': 2}
  ConvergenceWarning)


   54 | 00m06s |    0.82492 |         0.7016 |              2.1957 |        10.3729 | 
   55 | 00m06s |    0.83726 |         0.9320 |              2.1655 |        10.2255 | 
0.8372615039281706
{'n_estimators': 10.225490771619912, 'min_samples_split': 2.1655469479494975, 'max_features': 0.9319548705211711}


In [209]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
# 3分割交差検証を指定し、インスタンス化
kf = KFold(n_splits=3)
# 3分割し、交差検証
for  train_index,  test_index  in kf.split(trainX, Y):
    X_cv_train  = trainX.iloc[train_index]
    X_cv_test  = trainX.iloc[test_index]
    Y_cv_train  = Y.iloc[train_index]
    Y_cv_test  = Y.iloc[test_index]

    forest  =  RandomForestClassifier(
                    n_estimators=int(117.31343997905873),
                    min_samples_split=int(11.450953439652974),
                    max_features=0.6661230831444087,
                    random_state=17)
    forest.fit(X_cv_train, Y_cv_train)  

    predictions  = forest.predict(X_cv_test) 

    # acuuracyを表示
    print(round(accuracy_score(Y_cv_test,forest.predict(X_cv_test))*100,2))

81.82
83.84
84.18


In [210]:
from sklearn.model_selection import KFold

#  3分割交差検証を指定し、インスタンス化
kf  =  KFold(n_splits=3)
params  =  {
    'objective':  'binary',
    'learning_rate':  0.1,
    'num_leaves'  :  300
}

# kf.split(X_train.Ytrain)で、X_trainとY_trainを3分割し、交差検証をする
for  train_index,  test_index  in kf.split(trainX, Y):
    X_cv_train  = trainX.iloc[train_index]
    X_cv_test  = trainX.iloc[test_index]
    Y_cv_train  =  Y[train_index]
    Y_cv_test  =  Y[test_index]

    #  識別機を作成
    gbm  =  lgb.LGBMClassifier(objective='binary',
        num_leaves  =  300,
        learning_rate  =  0.1,
        )

    #  学習
    gbm.fit(X_cv_train, Y_cv_train,
    eval_set  =  [(X_cv_test,  Y_cv_test)],
    early_stopping_rounds=10,)

    #  予測し精度を表示
    Y_pred  = gbm.predict(X_cv_test,num_iteration=gbm.best_iteration_)
    print(round(accuracy_score(Y_cv_test,Y_pred)*100,2))

[1]	valid_0's binary_logloss: 0.615928
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's binary_logloss: 0.584909
[3]	valid_0's binary_logloss: 0.560557
[4]	valid_0's binary_logloss: 0.540186
[5]	valid_0's binary_logloss: 0.524069
[6]	valid_0's binary_logloss: 0.510362
[7]	valid_0's binary_logloss: 0.499594
[8]	valid_0's binary_logloss: 0.490339
[9]	valid_0's binary_logloss: 0.483095
[10]	valid_0's binary_logloss: 0.476975
[11]	valid_0's binary_logloss: 0.472212
[12]	valid_0's binary_logloss: 0.468094
[13]	valid_0's binary_logloss: 0.464488
[14]	valid_0's binary_logloss: 0.462399
[15]	valid_0's binary_logloss: 0.459138
[16]	valid_0's binary_logloss: 0.457916
[17]	valid_0's binary_logloss: 0.455514
[18]	valid_0's binary_logloss: 0.454854
[19]	valid_0's binary_logloss: 0.455383
[20]	valid_0's binary_logloss: 0.455334
[21]	valid_0's binary_logloss: 0.454541
[22]	valid_0's binary_logloss: 0.455194
[23]	valid_0's binary_logloss: 0.455177
[24]	valid_0's binary_loglo

In [211]:
pred_lr = lr.predict_proba(testX)
pred_forest = forest.predict_proba(testX)
pred_gbm = gbm.predict_proba(testX)

In [212]:
pred = pred_lr[:,1]*0.45 + pred_forest[:,1]*0.1 + pred_gbm[:,1]*0.45

In [213]:
pred_replace = [1 if i > 0.5 else 0 for i in pred]

In [214]:
# 提出用
gender_submission["Survived"] = pred_replace
gender_submission.to_csv('./submit/submit_ave19.csv',index=None)